In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 936, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 936 (delta 194), reused 112 (delta 103), pack-reused 665 (from 3)
Receiving objects: 100% (936/936), 2.28 MiB | 18.65 MiB/s, done.
Resolving deltas: 100% (576/576), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install "protobuf<5.0.0" --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.8 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydriv

In [3]:
import os
import sys
import pathlib
import kagglehub
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import tensorflow as tf

2025-12-11 21:29:34.198425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765488574.400777      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765488574.456531      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [5]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [6]:
from src.backtest.behavior_policy import run_behavior_policy_with_guards
from src.rl.state_builder import ensure_z_column, build_rl_dataset
from src.rl.cql_agent import CQLAgentTF, CQLConfig

In [7]:
# 1) Load final stacked dataset from 03_calibrate_and_stack
handle = "brian2000/calibrated-predictions"
filename = "calibrated_predictions"
dir_path = kagglehub.dataset_download(handle, force_download=True)
path = os.path.join(dir_path, filename + ".csv")
df =  pd.read_csv(path)

In [8]:
# 2) Ensure we have a z column
z_col ="z"
df = ensure_z_column(df, z_col=z_col, seed=42)

In [9]:
train_df=df[df['pred_source']=="oof"]
test_df=df[df['pred_source']=='test']

In [10]:
# 3) Run dumb behavior policy + guards for train df
df_policy = run_behavior_policy_with_guards(
    train_df,
    z_col=z_col,
    nav=1_000_000.0,
    allow_short=False,
    trading_enabled=True,
)

In [11]:
# 3) Run dumb behavior policy + guards for test
df_policy_test = run_behavior_policy_with_guards(
    test_df,
    z_col=z_col,
    nav=1_000_000.0,
    allow_short=False,
    trading_enabled=True,
)

In [12]:
# 4) Build RL training dataset (s, a, r, s')
extra_state_cols = ["news_flag", "disagreement"]
rl_df = build_rl_dataset(
    df_policy,
    z_col=z_col,
    reward_return_col="excess_return",
    action_col="action_weight_raw",
    weight_col="weight_after_guards",
    extra_state_cols=extra_state_cols,  # only if those exist
    lambda_risk=0.1,
)
rl_df.head()

,Date,ticker,reward,action,done,state_z,state_VIX_z,state_spread_z,state_weight_after_guards,state_news_flag,state_disagreement,next_state_z,next_state_VIX_z,next_state_spread_z,next_state_weight_after_guards,next_state_news_flag,next_state_disagreement
0,2012-01-03 00:00:00+00:00,AAL,-0.0,0.0,0,0.001625,-0.154233,0.864903,0.0,0,0.0,0.001682,-0.249174,0.345940,0.0,0.0,0.0
169,2012-01-04 00:00:00+00:00,AAL,0.0,0.0,0,0.001682,-0.249174,0.345940,0.0,0,0.0,0.001550,-0.342961,-0.972721,0.0,0.0,0.0
338,2012-01-05 00:00:00+00:00,AAL,0.0,0.0,0,0.001550,-0.342961,-0.972721,0.0,0,0.0,0.001613,-0.449838,-1.503982,0.0,0.0,0.0
507,2012-01-06 00:00:00+00:00,AAL,0.0,0.0,0,0.001613,-0.449838,-1.503982,0.0,0,0.0,0.001651,-0.397933,-0.289558,0.0,0.0,0.0
676,2012-01-09 00:00:00+00:00,AAL,-0.0,0.0,0,0.001651,-0.397933,-0.289558,0.0,0,0.0,0.001704,-0.446895,0.642269,0.0,0.0,0.0


In [13]:
rl_df = rl_df.dropna(subset=["state_z", "next_state_z"]).reset_index(drop=True)

In [14]:
#5) Extract state / next_state matrices and targets 
state_cols = [c for c in rl_df.columns if c.startswith("state_")]
next_state_cols = [c for c in rl_df.columns if c.startswith("next_state_")]

states = rl_df[state_cols].to_numpy().astype(np.float32)          # [N, state_dim]
next_states = rl_df[next_state_cols].to_numpy().astype(np.float32)

rewards = rl_df["reward"].to_numpy().astype(np.float32).reshape(-1, 1)  # [N,1]
dones = rl_df["done"].to_numpy().astype(np.float32).reshape(-1, 1)

# action is in weight units {0.0, 0.01, 0.02}. Map to indices {0,1,2}.
actions_w = rl_df["action"].to_numpy().astype(np.float32)
actions_idx = np.round(actions_w * 100).astype(np.int32).reshape(-1, 1)
# (0.00 -> 0, 0.01 -> 1, 0.02 -> 2)

state_dim = states.shape[1]
n_actions = 3

In [15]:
batch_size = 1024

dataset = tf.data.Dataset.from_tensor_slices(
    (states, actions_idx, rewards, next_states, dones)
)
dataset = dataset.shuffle(buffer_size=len(rl_df), reshuffle_each_iteration=True)
dataset = dataset.batch(batch_size)

I0000 00:00:1765488646.286729      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [16]:
#6) Training loop
config = CQLConfig(
    gamma=0.99,
    alpha=1.0,
    lr=1e-3,
    tau=0.005,
    n_actions=n_actions,
    hidden_dim=128,
)

agent = CQLAgentTF(state_dim=state_dim, config=config)

epochs = 100
steps_per_epoch = len(rl_df) // batch_size

for epoch in range(epochs):
    logs_epoch = {"loss": 0.0, "td_loss": 0.0, "cql_loss": 0.0}
    n_batches = 0

    for step, (s_b, a_b, r_b, ns_b, d_b) in enumerate(dataset):
        metrics = agent.train_step(s_b, a_b, r_b, ns_b, d_b)
        for k in logs_epoch:
            logs_epoch[k] += float(metrics[k])
        n_batches += 1

        if step >= steps_per_epoch:
            break

    agent.update_target()

    for k in logs_epoch:
        logs_epoch[k] /= max(n_batches, 1)

    print(
        f"Epoch {epoch:03d}: "
        f"loss={logs_epoch['loss']:.6f}, "
        f"td={logs_epoch['td_loss']:.6f}, "
        f"cql={logs_epoch['cql_loss']:.6f}"
    )

Epoch 000: loss=0.946339, td=0.077036, cql=0.869303
Epoch 001: loss=0.843874, td=0.075566, cql=0.768308
Epoch 002: loss=0.700480, td=0.078968, cql=0.621512
Epoch 003: loss=0.649291, td=0.083055, cql=0.566236
Epoch 004: loss=0.604755, td=0.079724, cql=0.525031
Epoch 005: loss=0.567754, td=0.076978, cql=0.490776
Epoch 006: loss=0.541989, td=0.075696, cql=0.466293
Epoch 007: loss=0.529376, td=0.077765, cql=0.451611
Epoch 008: loss=0.519511, td=0.076921, cql=0.442590
Epoch 009: loss=0.511690, td=0.076708, cql=0.434982
Epoch 010: loss=0.505971, td=0.075733, cql=0.430237
Epoch 011: loss=0.500249, td=0.075409, cql=0.424840
Epoch 012: loss=0.496886, td=0.074405, cql=0.422482
Epoch 013: loss=0.489742, td=0.073118, cql=0.416624
Epoch 014: loss=0.484048, td=0.071566, cql=0.412482
Epoch 015: loss=0.478631, td=0.070432, cql=0.408199
Epoch 016: loss=0.474808, td=0.069513, cql=0.405295
Epoch 017: loss=0.470815, td=0.068450, cql=0.402366
Epoch 018: loss=0.464576, td=0.067093, cql=0.397483
Epoch 019: l

In [17]:
# Save the Q-network that defines the policy
agent.q.save("/kaggle/working/cql_q_policy.keras")
# Save the df_policy dataset
df_policy_test.to_csv('df_policy_dataset.csv', index=False)

In [18]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!rm -rf /kaggle/working/__notebook__.ipynb

In [19]:
handle = "brian2000/cql-policy"
local_dataset_dir = f"/kaggle/working/"
current_date = datetime.today().strftime("%Y-%m-%d")
kagglehub.dataset_upload(handle, local_dataset_dir, version_notes= f"Dataset {current_date}")

Uploading Dataset https://www.kaggle.com/datasets/brian2000/cql-policy ...
Starting upload for file /kaggle/working/cql_q_policy.keras


Uploading: 100%|██████████| 91.8k/91.8k [00:00<00:00, 359kB/s]

Upload successful: /kaggle/working/cql_q_policy.keras (90KB)
Starting upload for file /kaggle/working/df_policy_dataset.csv



Uploading: 100%|██████████| 697M/697M [00:07<00:00, 95.7MB/s]

Upload successful: /kaggle/working/df_policy_dataset.csv (664MB)
Starting upload for file /kaggle/working/__notebook__.ipynb



Uploading: 100%|██████████| 332k/332k [00:00<00:00, 1.44MB/s]

Upload successful: /kaggle/working/__notebook__.ipynb (324KB)


Your dataset has been created.
Files are being processed...
See at: https://www.kaggle.com/datasets/brian2000/cql-policy
